# 資料處理Q&A

1. 先處理 NaN、inf 還先濾波??
    > 先處理 Nan、inf

2. ### 刪除Channel 還是刪除人??
    > 先刪除 Channel，不行再刪除人

In [ ]:
# import 必須資料
import os
import glob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 確認資料夾，確認VFT data 跟 .ipynb 沒放同一個資料夾
print(os.getcwd())

In [ ]:
# 找所有的csv名字
all_csv = glob.glob('VFT/*.csv')
print("length = ", len(all_csv))

# 計算沒有重複的名字，避免有些人重複多次
count_name = []
for banana in all_csv :
    real_name = banana.split('_')[-1].split('.')[0]
    if not real_name in count_name:
        count_name.append(real_name)

print('unique name = ', len(count_name))

# 看前五個
all_csv[:5]

## 先試試一個人
正常需使用`df = df.iloc[:, :11]`

`df = df.iloc[:, :9]` 是因為Ch4 經過考量捨棄，為捨棄前請全取

In [ ]:
select = 11
csv_name = all_csv[3]
df = pd.read_csv(csv_name)
df = df.iloc[:, :select]
df.columns

In [ ]:
# check null <必>
df.isna().sum()

# 畫圖看看

1. 用 `subplot()` 可以自動一格一格畫
2. 記得 `tight_layout`

In [ ]:
fig = plt.figure(figsize= (10, 10))


for i, cols in enumerate(df.columns[3:]):
    plt.subplot(int((select-3)/2), 2, i+1)
    plt.plot(df['Time_Host'], df[cols])
    plt.title(cols)
    plt.tight_layout()

fig.suptitle(f'{csv_name}',fontweight ="bold")

plt.show()

# 開始做data process
### 1. 看 NuN 、infinite 

In [ ]:
check_state = pd.DataFrame()

for i in all_csv:
    # 取名字
    filename = i.split('_')[-1]
    name = filename.split('.')[0] 
    dfs = pd.read_csv(i)
    # 取要的行
    dfs = dfs.iloc[:, :select]
    check_state[name + '_all'] = dfs.isna().sum() + np.isinf(dfs).sum()

check_state.tail(8)

### 看一下各Channel狀況(上面dataframe水平加起來)

> 太多Nan的就要去除掉

In [ ]:
check_state.sum(axis= 1)

### 去除Channel 4，請用drop

In [ ]:
check_state = pd.DataFrame()

for i in all_csv:
    filename = i.split('_')[-1]
    name = filename.split('.')[0] 
    dfs = pd.read_csv(i)
    dfs = dfs.iloc[:, :select]
    dfs.drop(columns=['CH4_Oxy', 'CH4_Deoxy'], inplace= True)
    check_state[name + '_all'] = dfs.isna().sum() + np.isinf(dfs).sum()

check_state.tail(8)

### 看一下每個人的狀況(上面dataframe垂直加起來)

> 太多Miss的就要去除掉

In [ ]:
rank = check_state.sum(axis= 0).sort_values(ascending= False)[:12]
rank

# NaN + inf > 1000 的人刪除

> 閥值自己訂


In [ ]:
print("Number of Data", len(all_csv))
bad = rank[rank >1000].index
for alls in all_csv:
    for bads in bad:
        name1 = bads.split("_")[0]

        if name1 in alls:
            print("Delect: ", alls)
            all_csv.remove(alls)

print("Number of Data", len(all_csv))

# 補值

> ### 就算少，還是不可以有 null, inf

1. ffill -> 最好用，補前值
2. bfill -> 補後值
3. `pandas.DataFrame.interpolate`  -> 1, 2, ?, 4, 5  $\rightarrow$ ? = 3



In [ ]:
# 隨便取一個看缺失值
df = pd.read_csv(all_csv[4])
dfs = df.iloc[:, :9]
print(np.isinf(dfs).sum())


### 確認補完後缺失值

In [ ]:
dfs[np.isinf(dfs)]= None
dfs = dfs.ffill(axis= 1)
np.isinf(dfs).sum() + dfs.isna().sum()

# 濾波
> 請先確認沒有miss值了

1. 四階butterword (可以更改)  -> 可自行查
```
from scipy import signal
b, c = signal.butter(4, 0.01, 'lowpass')
```

$W_n = \frac{f}{f_s/2}$ = $\frac{截止頻率}{取樣頻率/ 2}$

我們的

$\frac{0.01}{24/2} \approx 0.0083$

因為其實 0.01 ~ 0.015都可以，所以取0.012，$W_n就 0.01$

`b, c = signal.butter(4, Wn, 'lowpass')`

In [ ]:
# 開始濾波
from scipy import signal

# Wn = 0.01 
b, c = signal.butter(4, 0.01, 'lowpass')
print(len(b), len(c))

In [ ]:
# 濾CH1看看
CH1_HbO2_filter_a = signal.filtfilt(b, c, df.CH1_Oxy)
CH1_HbO2_filter_a

plt.plot(df.Time_Host, CH1_HbO2_filter_a)
plt.show()

### 請用for，不要一行一行濾

In [ ]:
# 全部一起濾波
need_filt = ['CH1_Oxy', 'CH1_Deoxy', 'CH2_Oxy', 'CH2_Deoxy', 'CH3_Oxy', 'CH3_Deoxy']

for need in need_filt:
    dfs[need] = signal.filtfilt(b, c, dfs[need])

# 看前五個
dfs.head()

### 她媽乾淨

In [ ]:
# 濾完了
plt.figure(figsize= (10, 10))

for i, cols in enumerate(need_filt):
    plt.subplot(int((select-3)/2), 2, i+1)
    plt.plot(dfs['Time_Host'], dfs[cols])
    plt.title(cols)
    plt.tight_layout()
plt.show()

# $\star$ 批量濾波 (試試函數能否套用於所有人)

> 上面都是以"一個人"為單位，現在"批量做"



In [ ]:
check_states = pd.DataFrame()
need_filt = ['CH1_Oxy', 'CH1_Deoxy', 'CH2_Oxy', 'CH2_Deoxy', 'CH3_Oxy', 'CH3_Deoxy']
b, c = signal.butter(4, 0.01, 'lowpass')

for i in all_csv:
    filename = i.split('_')[-1]
    name = filename.split('.')[0] 

    dfs = pd.read_csv(i)
    dfs = dfs.iloc[:, :9]

    dfs[np.isinf(dfs)]= None
    dfs = dfs.ffill(axis= 1)

    for needs in need_filt:
        dfs[needs] = signal.filtfilt(b, c, dfs[needs])

    # 看濾完波有沒有人不見    
    check_states[name + '_all'] = dfs.isna().sum()+ np.isinf(dfs).sum()



## 確認有濾好，沒有null

In [ ]:
# 以channel為單位
check_states.sum(axis= 1).sort_values(ascending= False)

In [ ]:
# 以人為單位
rank = check_states.sum(axis= 0).sort_values(ascending= False)[:10]
rank

# $\star$ MinMax -> 消除個體差異

> 消除個體差異

> $\frac{x - x_{min}}{x_{max} - x_{min}}$

In [ ]:
# minmax
dfs.iloc[:, 3:] = (dfs.iloc[:, 3:] - dfs.iloc[:, 3:].min())/(dfs.iloc[:, 3:].max()- dfs.iloc[:, 3:].min())
dfs

In [ ]:
# 畫出來
plt.figure(figsize= (10, 10))
for i, cols in enumerate(need_filt):
    plt.subplot(4, 2, i+1)
    plt.plot(dfs['Time_Host'], dfs[cols])
    plt.title(cols)
    plt.tight_layout()
plt.show()

---
---

# 整理一下整體流程 :  data preprocess + Lowpass filter + Minmax + to_csv

> 一段For (全人)
1. 讀取全部檔案，看是否有人名重複
2. 查看缺失值來判斷要刪除哪些Channel或是人
3. 刪完之後再補值

> 二段For (過濾太多缺失值)
4. 濾波(Filter)
5. minmax 消除個體差異
6. to_csv 存成csv，放於別的資料夾供之後使用


In [ ]:
# 若沒資料夾則創資料夾，有就略過
os.getcwd()
try:
    os.mkdir('Filtered')
except:
    pass

In [ ]:
# 找所有的csv名字
all_csv = glob.glob('VFT/*.csv')
print("length = ", len(all_csv))

# 計算沒有重複的名字，避免有些人重複多次
count_name = []
for banana in all_csv :
    real_name = banana.split('_')[-1].split('.')[0]
    if not real_name in count_name:
        count_name.append(real_name)

print('unique name = ', len(count_name))

# 看前五個
all_csv[:5]

In [ ]:
need_filt = ['CH1_Oxy', 'CH1_Deoxy', 'CH2_Oxy', 'CH2_Deoxy', 'CH3_Oxy', 'CH3_Deoxy']
a = 0

print("Number of Data", len(all_csv))
for files in all_csv:
    # 取人名
    filename = files.split('_')[-1]
    name = filename.split('.')[0] 

    #讀跟切(取下要的就好)
    dfs = pd.read_csv(files)
    dfs = dfs.iloc[:, :11]
    dfs.drop(columns=['CH4_Oxy', 'CH4_Deoxy'], inplace= True)



    null_count = dfs.isna().sum() + np.isinf(dfs).sum()
    
    # 刪除缺失值大於1000的人
    many_null = len(null_count[null_count > 1000])

    if many_null > 0:
        print(name, "'s data is not good to use")
        continue
    
    a = a + 1


    # inf to nan to ffill
    dfs[np.isinf(dfs)]= None
    dfs = dfs.ffill(axis= 1)

    # 濾波
    b, c = signal.butter(4, 0.01, 'lowpass')
    for needs in need_filt:
        dfs[needs] = signal.filtfilt(b, c, dfs[needs])

    # minmax
    dfs.iloc[:, 3:] = (dfs[need_filt] - dfs[need_filt].min())/(dfs[need_filt].max()- dfs[need_filt].min())
        
    dfs.to_csv('Filtered\\'+ "F_" + filename, index= False)

print("Number of Data ", a)

# 讀取濾波完的 .csv，並畫圖
> ### time 變成 index 可以直接用loc切

In [ ]:
alls = glob.glob('Filtered/*.csv')
print(len(alls))
alls[:5]

In [ ]:
pd.read_csv(alls[5])

In [ ]:
# 看個資料型態
check_df = pd.read_csv(alls[25], index_col= 'Time_Host').drop(columns= ['Time_Arduino', 'easingdata'])
print(check_df.dtypes)

check_df.head()

# 畫圖

### matplotlib.pyplot 自己練

重點:
1. `facecolor=sns.color_palette('Paired')[1]`  -> 自己配色會比我阿嬤還醜
2. `bbox=dict(boxstyle="round",ec=(1., 0.5, 0.5),fc=(1., 0.8, 0.8),)1` -> bbox 可以唱bbox

### 階段

* 第一階段: 0 ~ 30  -> rest
* 第二階段: 30 ~ 90  -> Task
* 第三階段: 90 ~ 160  -> recovery

In [ ]:
from matplotlib.lines import Line2D
custom_lines = [Line2D([0], [0], color='blue', lw=4),
                Line2D([0], [0], color='red', lw=4)]



In [ ]:
stage1 = 30
stage2 = 90
stage3 = 160
text_size = 25

plt.figure(figsize= (18, 14))
for i in range(len(check_df.columns)//2):
    plt.subplot(3, 1, i+1)

    # 第一階段
    plt.plot(check_df.loc[:stage1].index, check_df.loc[:stage1].iloc[:, 2*i], c= 'b',  linewidth=3.0, label= 'Rest_Oxy')
    plt.plot(check_df.loc[:stage1].index, check_df.loc[:stage1].iloc[:, 2*i + 1], c= 'r',  linewidth=3.0, label= 'Rest_deoxy')

    plt.axvspan(0, stage1, facecolor = sns.color_palette('Paired')[0], alpha=0.5)
    plt.vlines(stage1, -0.1, 1.3, linestyles= '--', colors= 'black',  linewidth=2.0)
    plt.text(stage1/2, 1.2, "Rest", size= text_size, ha="center", va= 'center', bbox=dict(boxstyle="round", ec=(1., 0.5, 0.5), fc=(1., 0.8, 0.8)))

    # 第二階段
    plt.plot(check_df.loc[stage1:stage2].index, check_df.loc[stage1:stage2].iloc[:, 2*i], c= 'b', linewidth=3.0, label= 'Task_Oxy')
    plt.plot(check_df.loc[stage1:stage2].index, check_df.loc[stage1:stage2].iloc[:, 2*i + 1], c= 'r', linewidth=3.0, label= 'Task_Deoxy')
    
    plt.axvspan(stage1, stage2, facecolor=sns.color_palette('Paired')[1], alpha=0.5)
    plt.vlines(stage2, -0.1, 1.3, linestyles= '--', colors= 'black',  linewidth=2.0)
    plt.text((stage2 + stage1)/2, 1.2, 'Task', size= text_size, ha="center", va= 'center', bbox=dict(boxstyle="round",ec=(1., 0.5, 0.5),fc=(1., 0.8, 0.8),))

    # 第三階段
    plt.plot(check_df.loc[stage2:stage3].index, check_df.loc[stage2:stage3].iloc[:, 2*i], c= 'b', linewidth=3.0, label= 'Recovery_Oxy')
    plt.plot(check_df.loc[stage2:stage3].index, check_df.loc[stage2:stage3].iloc[:, 2*i + 1], c= 'r', linewidth=3.0, label= 'Recovery_Oxy')

    plt.axvspan(stage2, stage3, facecolor=sns.color_palette('Paired')[2], alpha=0.5)
    plt.text((stage3 + stage2)/2, 1.2, 'Recovery', size= text_size, ha="center", va= 'center', bbox=dict(boxstyle="round",ec=(1., 0.5, 0.5),fc=(1., 0.8, 0.8),))

    plt.title(f'CH{i+1}_blood_info', fontdict={'fontsize': 24})
    plt.legend(custom_lines, ['Oxygen_Blood', 'Deoxygen_Blood'])
    plt.tight_layout(pad= 3)
plt.show()